In [1]:
#Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from citipy import citipy
import requests
from datetime import datetime
from config import weather_api_key
from scipy.stats import linregress
import matplotlib.pyplot as plt

url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [2]:
# Create a set of 1,500 random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs 

coordinates = list(lat_lngs)

# Create a list for holding the cities.

# Get the nearest city using the citipy module.
cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

593

In [ ]:
# Perform an API call with the OpenWeatherMap.

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")


city_data = []
record_count = 1
set_count = 1
    
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
print(len(city_data))

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | rikitea
Processing Record 2 of Set 1 | jamestown
Processing Record 3 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 4 of Set 1 | along
Processing Record 5 of Set 1 | esperance
Processing Record 6 of Set 1 | berlevag
Processing Record 7 of Set 1 | attawapiskat
City not found. Skipping...
Processing Record 8 of Set 1 | lompoc
Processing Record 9 of Set 1 | mataura
Processing Record 10 of Set 1 | boddam
Processing Record 11 of Set 1 | dikson
Processing Record 12 of Set 1 | atar
Processing Record 13 of Set 1 | sorland
Processing Record 14 of Set 1 | chipinge
Processing Record 15 of Set 1 | sterling
Processing Record 16 of Set 1 | inirida
Processing Record 17 of Set 1 | nanortalik
Processing Record 18 of Set 1 | laguna de perlas
Processing Record 19 of Set 1 | tiksi
Processing Record 20 of Set 1 | longyearbyen
Processing Record 21 of Set 1 | albany
Processing Record 22 of Se

Processing Record 39 of Set 4 | hofn
Processing Record 40 of Set 4 | creteni
Processing Record 41 of Set 4 | san rafael
Processing Record 42 of Set 4 | camana
Processing Record 43 of Set 4 | malangwa
Processing Record 44 of Set 4 | kailua
Processing Record 45 of Set 4 | buchanan
Processing Record 46 of Set 4 | darhan
Processing Record 47 of Set 4 | abnub
Processing Record 48 of Set 4 | kemijarvi
Processing Record 49 of Set 4 | saldanha
Processing Record 50 of Set 4 | alta floresta
Processing Record 1 of Set 5 | burica
City not found. Skipping...
Processing Record 2 of Set 5 | ikom
Processing Record 3 of Set 5 | norman wells
Processing Record 4 of Set 5 | tanakpur
Processing Record 5 of Set 5 | mayo
Processing Record 6 of Set 5 | touros
Processing Record 7 of Set 5 | tucuru
Processing Record 8 of Set 5 | onega
Processing Record 9 of Set 5 | xuddur
Processing Record 10 of Set 5 | pacific grove
Processing Record 11 of Set 5 | innisfail
Processing Record 12 of Set 5 | san carlos de bariloc

Processing Record 27 of Set 8 | kizema
Processing Record 28 of Set 8 | achisay
City not found. Skipping...
Processing Record 29 of Set 8 | dubti
Processing Record 30 of Set 8 | doctor pedro p. pena
City not found. Skipping...
Processing Record 31 of Set 8 | cidreira
Processing Record 32 of Set 8 | manturovo
Processing Record 33 of Set 8 | kosonsoy
Processing Record 34 of Set 8 | kantabanji
Processing Record 35 of Set 8 | sisophon
Processing Record 36 of Set 8 | tuatapere
Processing Record 37 of Set 8 | cloquet
Processing Record 38 of Set 8 | agadez
Processing Record 39 of Set 8 | dullewala
Processing Record 40 of Set 8 | kloulklubed
Processing Record 41 of Set 8 | huoqiu
Processing Record 42 of Set 8 | cervo
Processing Record 43 of Set 8 | katherine
Processing Record 44 of Set 8 | saint-augustin
Processing Record 45 of Set 8 | sainte-maxime
Processing Record 46 of Set 8 | abu dhabi
Processing Record 47 of Set 8 | uyuni
Processing Record 48 of Set 8 | luderitz
Processing Record 49 of Se

Processing Record 14 of Set 12 | copiapo
Processing Record 15 of Set 12 | si sa ket
Processing Record 16 of Set 12 | kilindoni
Processing Record 17 of Set 12 | loiza
Processing Record 18 of Set 12 | amapa
Processing Record 19 of Set 12 | tuy hoa
Processing Record 20 of Set 12 | benguela
Processing Record 21 of Set 12 | barra do garcas
Processing Record 22 of Set 12 | umm kaddadah
Processing Record 23 of Set 12 | bambanglipuro
Processing Record 24 of Set 12 | kamenka
Processing Record 25 of Set 12 | anadyr
Processing Record 26 of Set 12 | muggia
Processing Record 27 of Set 12 | kalabo
Processing Record 28 of Set 12 | aksarka
Processing Record 29 of Set 12 | maceio
Processing Record 30 of Set 12 | goure
